# 06 Evaluation (MSE + QLIKE)

Compare core model families evaluated with both `MSE` and `QLIKE`:
- GARCH(1,1)-t baseline
- Pure LSTM (MSE-trained)
- Pure GRU (MSE-trained)
- Hybrid LSTM (GARCH + LSTM, MSE-trained)
- Hybrid GRU (GARCH + GRU, MSE-trained)

If residual-hybrid outputs from notebook 05a exist, they are included automatically.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.evaluation import evaluate_forecasts


In [ ]:
pred_dir = PROJECT_ROOT / "reports" / "predictions"
required_files = [
    pred_dir / "garch_baseline_predictions.csv",
    pred_dir / "pure_lstm_predictions.csv",
    pred_dir / "pure_gru_predictions.csv",
    pred_dir / "hybrid_lstm_predictions.csv",
    pred_dir / "hybrid_gru_predictions.csv",
]
optional_files = [
    pred_dir / "hybrid_residual_lstm_predictions.csv",
    pred_dir / "hybrid_residual_gru_predictions.csv",
]

missing = [str(path) for path in required_files if not path.exists()]
if missing:
    raise FileNotFoundError(
        "Missing required prediction files. Run notebooks 02, 03, 04, and 05 first:\n" + "\n".join(missing)
    )

existing_optional = [path for path in optional_files if path.exists()]
prediction_files = required_files + existing_optional

print("Using prediction files:")
for path in prediction_files:
    print(" -", path.name)

pred_frames = [pd.read_csv(path, parse_dates=["date"]) for path in prediction_files]
all_predictions = pd.concat(pred_frames, ignore_index=True)

all_predictions.head()


In [ ]:
metrics_by_model = evaluate_forecasts(
    all_predictions,
    group_cols=["variant", "architecture"],
)

metrics_by_model = metrics_by_model.sort_values(["qlike", "mse"]).reset_index(drop=True)
metrics_by_model


In [ ]:
overall_metrics = evaluate_forecasts(all_predictions)
overall_metrics


In [ ]:
plot_df = metrics_by_model.copy()
plot_df["model_name"] = plot_df["variant"].str.title() + " " + plot_df["architecture"].str.upper()

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
sns.barplot(data=plot_df, x="model_name", y="mse", ax=axes[0], color="steelblue")
sns.barplot(data=plot_df, x="model_name", y="qlike", ax=axes[1], color="indianred")

axes[0].set_title("MSE by Model")
axes[1].set_title("QLIKE by Model")
axes[0].tick_params(axis="x", rotation=25)
axes[1].tick_params(axis="x", rotation=25)
axes[0].grid(alpha=0.25)
axes[1].grid(alpha=0.25)

plt.tight_layout()
plt.show()


In [ ]:
plot_df_no_hybrid = plot_df[plot_df["variant"] != "hybrid_residual"].copy()
if plot_df_no_hybrid.empty:
    raise ValueError("No rows found after excluding hybrid_residual for filtered QLIKE plot.")

fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(data=plot_df_no_hybrid, x="model_name", y="qlike", ax=ax, color="teal")
ax.set_title("QLIKE by Model (Excluding Residual-Hybrid)")
ax.tick_params(axis="x", rotation=25)
ax.grid(alpha=0.25)
plt.tight_layout()
plt.show()


In [ ]:
# Load VIX and build model-level daily panel for diagnostics plots.
vix_cache_path = PROJECT_ROOT / "data" / "processed" / "vix_daily.csv"
start_date = (all_predictions["date"].min() - pd.Timedelta(days=30)).strftime("%Y-%m-%d")
end_date = (all_predictions["date"].max() + pd.Timedelta(days=7)).strftime("%Y-%m-%d")

cached_vix = None
if vix_cache_path.exists():
    cached_vix = pd.read_csv(vix_cache_path, parse_dates=["date"])

try:
    vix_raw = yf.download("^VIX", start=start_date, end=end_date, auto_adjust=False, progress=False)
    if vix_raw.empty:
        raise ValueError("VIX download returned empty dataframe.")

    if isinstance(vix_raw.columns, pd.MultiIndex):
        vix_raw.columns = [col[0] for col in vix_raw.columns]

    vix_raw = vix_raw.rename_axis("date").reset_index()
    vix_raw.columns = [str(c).strip().lower().replace(" ", "_") for c in vix_raw.columns]
    close_col = "adj_close" if "adj_close" in vix_raw.columns else "close"

    vix_df = vix_raw[["date", close_col]].rename(columns={close_col: "vix_close"})
    vix_df["date"] = pd.to_datetime(vix_df["date"], errors="raise").dt.tz_localize(None)
    vix_df = vix_df.dropna(subset=["vix_close"]).sort_values("date").reset_index(drop=True)
    vix_df.to_csv(vix_cache_path, index=False)
except Exception as exc:
    if cached_vix is None or cached_vix.empty:
        raise RuntimeError("Unable to load VIX data from yfinance and no cache exists.") from exc
    print(f"Using cached VIX data due to download error: {exc}")
    vix_df = cached_vix.copy()

model_daily = (
    all_predictions
    .groupby(["date", "variant", "architecture"], as_index=False)
    .agg(y_true_var=("y_true_var", "mean"), y_pred_var=("y_pred_var", "mean"))
)
model_daily["model_name"] = model_daily["variant"].str.title() + " " + model_daily["architecture"].str.upper()

model_vix = model_daily.merge(vix_df, on="date", how="inner")
model_vix = model_vix.sort_values(["model_name", "date"]).reset_index(drop=True)
model_vix["sq_error"] = (model_vix["y_true_var"] - model_vix["y_pred_var"]) ** 2

ROLLING_MSE_WINDOW = 63
model_vix["rolling_mse"] = model_vix.groupby("model_name")["sq_error"].transform(
    lambda s: s.rolling(window=ROLLING_MSE_WINDOW, min_periods=ROLLING_MSE_WINDOW).mean()
)

print(f"Model+VIX merged rows: {len(model_vix):,}")
model_vix.head()


In [ ]:
# Plot VIX vs forecast variance for each model.
model_names = sorted(model_vix["model_name"].unique())
ncols = 2
nrows = (len(model_names) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 3.8 * nrows), sharex=True)
axes = np.array(axes).reshape(-1)

for ax, model_name in zip(axes, model_names):
    d = model_vix[model_vix["model_name"] == model_name]
    ax.plot(d["date"], d["y_pred_var"], color="tab:blue", linewidth=1.0)
    ax.set_ylabel("Forecast Var", color="tab:blue")
    ax.tick_params(axis="y", labelcolor="tab:blue")
    ax.grid(alpha=0.2)

    ax2 = ax.twinx()
    ax2.plot(d["date"], d["vix_close"], color="tab:orange", alpha=0.7, linewidth=1.0)
    ax2.set_ylabel("VIX", color="tab:orange")
    ax2.tick_params(axis="y", labelcolor="tab:orange")
    ax.set_title(model_name)

for ax in axes[len(model_names):]:
    ax.axis("off")

fig.suptitle("VIX vs Forecast Variance by Model", y=1.01)
plt.tight_layout()
plt.show()


In [ ]:
# Plot rolling MSE (63d) with VIX for each model.
rolling_plot_df = model_vix.dropna(subset=["rolling_mse"]).copy()
if rolling_plot_df.empty:
    raise ValueError("Rolling MSE dataframe is empty; reduce ROLLING_MSE_WINDOW or check data coverage.")

model_names = sorted(rolling_plot_df["model_name"].unique())
ncols = 2
nrows = (len(model_names) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 3.8 * nrows), sharex=True)
axes = np.array(axes).reshape(-1)

for ax, model_name in zip(axes, model_names):
    d = rolling_plot_df[rolling_plot_df["model_name"] == model_name]
    ax.plot(d["date"], d["rolling_mse"], color="tab:green", linewidth=1.0)
    ax.set_ylabel("Rolling MSE", color="tab:green")
    ax.tick_params(axis="y", labelcolor="tab:green")
    ax.grid(alpha=0.2)

    ax2 = ax.twinx()
    ax2.plot(d["date"], d["vix_close"], color="tab:orange", alpha=0.7, linewidth=1.0)
    ax2.set_ylabel("VIX", color="tab:orange")
    ax2.tick_params(axis="y", labelcolor="tab:orange")
    ax.set_title(model_name)

for ax in axes[len(model_names):]:
    ax.axis("off")

fig.suptitle("Rolling 63-Day MSE vs VIX by Model", y=1.01)
plt.tight_layout()
plt.show()


In [ ]:
# Calibration-style comparison: model forecast vs realized variance, with VIX-implied variance baseline.
model_vix["vix_implied_var_1d"] = ((model_vix["vix_close"] / 100.0) ** 2) / 252.0
for col in ["y_true_var", "y_pred_var", "vix_implied_var_1d"]:
    model_vix[col] = model_vix[col].clip(lower=1e-12)

comparison_rows = []
for model_name, d in model_vix.groupby("model_name"):
    comparison_rows.append(
        {
            "model_name": model_name,
            "corr_model_vs_realized": d["y_pred_var"].corr(d["y_true_var"]),
            "corr_vix_vs_realized": d["vix_implied_var_1d"].corr(d["y_true_var"]),
            "mse_model": ((d["y_true_var"] - d["y_pred_var"]) ** 2).mean(),
            "mse_vix": ((d["y_true_var"] - d["vix_implied_var_1d"]) ** 2).mean(),
        }
    )

comparison_df = pd.DataFrame(comparison_rows).sort_values("mse_model").reset_index(drop=True)
comparison_df


In [ ]:
model_names = sorted(model_vix["model_name"].unique())
ncols = 2
nrows = (len(model_names) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 4.2 * nrows), sharex=False, sharey=False)
axes = np.array(axes).reshape(-1)

for ax, model_name in zip(axes, model_names):
    d = model_vix[model_vix["model_name"] == model_name].copy()
    d["true_bin"] = pd.qcut(d["y_true_var"], q=20, duplicates="drop")
    cal = (
        d.groupby("true_bin", observed=True)
        .agg(
            mean_true=("y_true_var", "mean"),
            mean_model=("y_pred_var", "mean"),
            mean_vix=("vix_implied_var_1d", "mean"),
        )
        .dropna()
        .sort_values("mean_true")
    )

    ax.plot(cal["mean_true"], cal["mean_model"], marker="o", linewidth=1.5, label="Model")
    ax.plot(cal["mean_true"], cal["mean_vix"], marker="s", linewidth=1.3, label="VIX Implied")

    min_val = min(cal["mean_true"].min(), cal["mean_model"].min(), cal["mean_vix"].min())
    max_val = max(cal["mean_true"].max(), cal["mean_model"].max(), cal["mean_vix"].max())
    ax.plot([min_val, max_val], [min_val, max_val], linestyle="--", color="gray", linewidth=1.0, label="Ideal")

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title(model_name)
    ax.set_xlabel("Mean Realized Variance (by bin)")
    ax.set_ylabel("Mean Predicted Variance")
    ax.grid(alpha=0.2)

for ax in axes[len(model_names):]:
    ax.axis("off")

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=3, frameon=False)
fig.suptitle("Calibration vs Realized Variance: Model Forecast vs VIX Implied", y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Realized-vol diagnostics (21-day annualized): model forecast vs realized volatility.
REALIZED_VOL_WINDOW = 21
model_vix["realized_vol_21d"] = np.sqrt(
    model_vix.groupby("model_name")["y_true_var"].transform(
        lambda s: s.rolling(window=REALIZED_VOL_WINDOW, min_periods=REALIZED_VOL_WINDOW).mean()
    ) * 252.0
)
model_vix["forecast_vol_21d"] = np.sqrt(
    model_vix.groupby("model_name")["y_pred_var"].transform(
        lambda s: s.rolling(window=REALIZED_VOL_WINDOW, min_periods=REALIZED_VOL_WINDOW).mean()
    ) * 252.0
)

rv_diag = model_vix.dropna(subset=["realized_vol_21d", "forecast_vol_21d"]).copy()
if rv_diag.empty:
    raise ValueError("Realized-vol diagnostic dataframe is empty. Check data availability.")

rv_summary_rows = []
for model_name, d in rv_diag.groupby("model_name"):
    rv_summary_rows.append(
        {
            "model_name": model_name,
            "corr_forecast_vs_realized_vol": d["forecast_vol_21d"].corr(d["realized_vol_21d"]),
            "mae_forecast_vs_realized_vol": (d["forecast_vol_21d"] - d["realized_vol_21d"]).abs().mean(),
        }
    )

rv_summary = pd.DataFrame(rv_summary_rows).sort_values("mae_forecast_vs_realized_vol").reset_index(drop=True)
rv_summary


In [ ]:
# Plot forecast vol vs realized vol by model.
model_names = sorted(rv_diag["model_name"].unique())
ncols = 2
nrows = (len(model_names) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 3.8 * nrows), sharex=True)
axes = np.array(axes).reshape(-1)

for ax, model_name in zip(axes, model_names):
    d = rv_diag[rv_diag["model_name"] == model_name]
    ax.plot(d["date"], d["realized_vol_21d"], color="tab:gray", linewidth=1.1, label="Realized Vol (21d)")
    ax.plot(d["date"], d["forecast_vol_21d"], color="tab:blue", linewidth=1.1, label="Forecast Vol (21d)")
    ax.set_title(model_name)
    ax.set_ylabel("Annualized Vol")
    ax.grid(alpha=0.2)
    ax.legend(loc="upper right")

for ax in axes[len(model_names):]:
    ax.axis("off")

fig.suptitle("Forecast vs Realized Volatility (21-Day, Annualized)", y=1.01)
plt.tight_layout()
plt.show()


In [ ]:
# Plot rolling MSE with realized vol for each model.
rolling_rv_df = rv_diag.dropna(subset=["rolling_mse"]).copy()
if rolling_rv_df.empty:
    raise ValueError("Rolling MSE + realized-vol dataframe is empty.")

model_names = sorted(rolling_rv_df["model_name"].unique())
ncols = 2
nrows = (len(model_names) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(16, 3.8 * nrows), sharex=True)
axes = np.array(axes).reshape(-1)

for ax, model_name in zip(axes, model_names):
    d = rolling_rv_df[rolling_rv_df["model_name"] == model_name]
    ax.plot(d["date"], d["rolling_mse"], color="tab:green", linewidth=1.0)
    ax.set_ylabel("Rolling MSE", color="tab:green")
    ax.tick_params(axis="y", labelcolor="tab:green")
    ax.grid(alpha=0.2)

    ax2 = ax.twinx()
    ax2.plot(d["date"], d["realized_vol_21d"], color="tab:purple", alpha=0.7, linewidth=1.0)
    ax2.set_ylabel("Realized Vol (21d)", color="tab:purple")
    ax2.tick_params(axis="y", labelcolor="tab:purple")
    ax.set_title(model_name)

for ax in axes[len(model_names):]:
    ax.axis("off")

fig.suptitle("Rolling 63-Day MSE vs Realized Volatility", y=1.01)
plt.tight_layout()
plt.show()


In [ ]:
summary_path = PROJECT_ROOT / "reports" / "predictions" / "evaluation_metrics_mse_qlike.csv"
metrics_by_model.to_csv(summary_path, index=False)
print(f"Saved metric summary: {summary_path}")
